Copyright (c) Snap Inc. 2020. This sample code is made available by Snap Inc. for informational purposes only. It is provided as-is, without warranty of any kind, express or implied, including any warranties of merchantability, fitness for a particular purpose, or non-infringement. In no event will Snap Inc. be liable for any damages arising from the sample code or your use thereof.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import hashlib
import spacy
import random
import os
import re
import json
from collections import OrderedDict
from operator import itemgetter
import itertools
from spacy.lang.en.stop_words import STOP_WORDS
import string
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import sys
sys.path.append('../')

from tf_idf_vectorizer import *
# from utils.snap_preprocessed_df_handle import *
from utils.EstimatorSelectionHelper import EstimatorSelectionHelper
from utils.classifier_setup import *

In [ ]:
main_df = pd.read_csv('../../data/bugrepo/EclipsePlatform/eclipse_platform.csv')

In [ ]:
train_df = pd.read_csv('../../data/bugrepo/EclipsePlatform/train.csv').dropna().reset_index(drop=True)
test_df = pd.read_csv('../../data/bugrepo/EclipsePlatform/test.csv').dropna().reset_index(drop=True)

In [ ]:
train_df['Duplicate'] = train_df.Duplicate.str.split(';')
test_df['Duplicate'] = test_df.Duplicate.str.split(';')

In [ ]:
main_df

In [ ]:
train_dict = {'id1':[], 'id2':[]}

for index, row in train_df.iterrows():
    dups = row['Duplicate']
    for id2 in dups:
        train_dict['id1'].append(row['Issue_id'])
        train_dict['id2'].append(id2)

In [ ]:
test_dict = {'id1':[], 'id2':[]}

for index, row in test_df.iterrows():
    dups = row['Duplicate']
    for id2 in dups:
        test_dict['id1'].append(row['Issue_id'])
        test_dict['id2'].append(id2)

In [ ]:
train_df = pd.DataFrame(train_dict)
test_df = pd.DataFrame(test_dict)

In [ ]:
def get_groups(row):
    group1 = main_df[main_df['Issue_id']==int(row['id1'])]['Component'].values[0]
    group2 = main_df[main_df['Issue_id']==int(row['id2'])]['Component'].values[0]
    
    if group1!=group2:
        return None
    else:
        return group1

In [ ]:
def get_final_dict(group_df_train, train_df):
    
    final_train_dict = {'id1':[],'id2':[],'dup_issue':[],'dup_group':[],'title1':[],'title2':[]}
    
    for name, group in group_df_train:
        for index, row in group.iterrows():
            final_train_dict['id1'].append(row['id1'])
            final_train_dict['id2'].append(row['id2'])
            final_train_dict['dup_issue'].append(True)
            final_train_dict['dup_group'].append(True)
            final_train_dict['title1'].append(main_df[main_df['Issue_id']==int(row['id1'])]['Title'].values[0])
            final_train_dict['title2'].append(main_df[main_df['Issue_id']==int(row['id2'])]['Title'].values[0])

        main_same_group = main_df[main_df['Component']==name]
        all_ids = []
        all_ids.extend(list(group['id1']))
        all_ids.extend(list(group['id2']))
        all_ids = list(set(all_ids))
        not_in_group_df = main_same_group[main_same_group["Issue_id"].apply(lambda x: x not in all_ids)]
        not_in_group_df = not_in_group_df.sample(n=len(all_ids))
        k=int(0.2*len(all_ids))
        same_group_diff_bug = random.sample(set(itertools.product(all_ids, list(not_in_group_df['Issue_id']))), k)

        for (id1, id2) in same_group_diff_bug:
            final_train_dict['id1'].append(id1)
            final_train_dict['id2'].append(id2)
            final_train_dict['dup_issue'].append(False)
            final_train_dict['dup_group'].append(True)
            final_train_dict['title1'].append(main_df[main_df['Issue_id']==int(id1)]['Title'].values[0])
            final_train_dict['title2'].append(main_df[main_df['Issue_id']==int(id2)]['Title'].values[0])

        remaining_k = int(0.8*len(all_ids))
        main_not_same_group = train_df[train_df['common_group']!=name]
        all_ids_not_same = []
        all_ids_not_same.extend(list(group['id1']))
        all_ids_not_same.extend(list(group['id2']))
        all_ids_not_same = list(set(all_ids_not_same))
        diff_group_diff_bug = random.sample(set(itertools.product(all_ids, all_ids_not_same)), remaining_k)

        for (id1, id2) in diff_group_diff_bug:
            final_train_dict['id1'].append(id1)
            final_train_dict['id2'].append(id2)
            final_train_dict['dup_issue'].append(False)
            final_train_dict['dup_group'].append(False)
            final_train_dict['title1'].append(main_df[main_df['Issue_id']==int(id1)]['Title'].values[0])
            final_train_dict['title2'].append(main_df[main_df['Issue_id']==int(id2)]['Title'].values[0])
            
    
    return final_train_dict
  

In [ ]:
train_df['common_group'] = train_df.apply(get_groups, axis=1)
test_df['common_group'] = test_df.apply(get_groups, axis=1)

In [ ]:
group_df_train = train_df.groupby('common_group')
group_df_test = test_df.groupby('common_group')

In [ ]:
final_test_dict = get_final_dict(group_df_test, test_df)
final_train_dict = get_final_dict(group_df_train, train_df)

In [ ]:
final_train = pd.DataFrame(final_train_dict)
final_test = pd.DataFrame(final_test_dict)

In [ ]:
final_train.to_pickle('../../data/dataframes/df_train_bugrepo_eclipse.pkl')
final_test.to_pickle('../../data/dataframes/df_test_bugrepo_eclipse.pkl')